## PyCon UK '23 workshop: Turn Satellite Images into Digital Art with Python!
---

In [ ]:
# Set-up
%config Completer.use_jedi = False

In [ ]:
# Set-up
%pip install -q scipy
%pip install -q matplotlib

In [ ]:
# Set-up
from time import time
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

In [ ]:
# Load satellite image jpg
raw_jpg = Image.open("Earth.jpg").convert("RGB")

In [ ]:
# Convert to numpy array
array_uint8 = np.array(raw_jpg) 

In [ ]:
# Make function that generates a colour swatch of a specific pixel of the image data
def pixel_colour_swatch(row, column):
    RGB_colour = tuple(array_uint8[row][column].tolist())
    print(f"The pixel at ({row}, {column}) has RGB of {RGB_colour}.")
    return Image.new("RGB", (50,50), RGB_colour)

In [ ]:
# Call the swatch function
pixel_colour_swatch(10, 10)

In [ ]:
# Assign the dimensions of the array_uint8 object to variables, then check these
width, height, depth = original_shape = array_uint8.shape
original_shape

In [ ]:
# Test the condition that the value of the depth variable is 3 (as would be the case of the image colour mode is RGB)
# Reshape the array_uint8 object into width * height no. of rows, and depth no. of columns (which is expected to always be 3)
assert depth == 3
reshaped_array_uint8 = np.reshape(array_uint8, (width * height, depth))

In [ ]:
# Check the dimensions of array_unint8
reshaped_array_uint8.shape
#reshaped_array_uint8[:5]

In [ ]:
# OPTIONAL: Calculate the number of colours in the original satellite image data
original_n_colours = np.unique(reshaped_array_uint8, axis=0).shape[0]
original_n_colours

In [ ]:
# Convert to the default 8 bits integer coding to floats and divide by 255 so the floats are in the range [0-1] (normalised)
reshaped_array_float01range = np.array(reshaped_array_uint8, dtype=np.float64) / 255

In [ ]:
# Make a copy of the prepared image data, image_array, to work on
image_array = reshaped_array_float01range.copy()

In [ ]:
# Choose the number of clusters, i.e. colours, that the image data will be grouped/reduced into
n_colours = 6

In [ ]:
# Train K-Means clustering model on small sub-sample of the image data
print("Fitting model on a small sub-sample of the data")
t0 = time()
image_array_sample = shuffle(image_array, random_state=0, n_samples=1_000)
kmeans = KMeans(n_clusters=n_colours, n_init="auto", random_state=0).fit(
    image_array_sample
)
print(f"done in {time() - t0:0.3f}s.")

In [ ]:
# Get labels for all points - use the kmeans trained model to predict which of the n clusters each pixel belongs to, i.e. assign each pixel a label, from 1 to n_colors
print("Predicting colour indices on the full image (k-means)")
t0 = time()
labels = kmeans.predict(image_array)
print(f"done in {time() - t0:0.3f}s.")

In [ ]:
# Import a module that enables useful color functionality 
import matplotlib.colors as mcolours

In [ ]:
# Check out available 'named colours' and what their RGB value is: https://matplotlib.org/stable/gallery/color/named_colors.html#css-colors
mcolours.to_rgb("tomato")

In [ ]:
# Write function for creating custom colour palettes, referred to here as a 'codebook'
# FYI can also use a list comprehension for the main body of the function, e.g. `codebook = np.array([mcolors.to_rgb(i) for i in args])`

def make_codebook(*args):
    codebook = []
    for i in args:
        codebook.append(mcolours.to_rgb(i))
    codebook = np.array(codebook)
    return codebook

In [ ]:
# Use the `make_codebook` function to create a custom colour palette with which to recreate an updated satellite image as digital artwork
# NOTE: Ensure to pass the same number of 'named colors' arguments as the number of 'n_colors' specified earlier (6 in the prepared example)

codebook_6colours = make_codebook("mediumspringgreen", "cornflowerblue", "thistle", "coral", "khaki",  "darkturquoise")

In [ ]:
# Write function to recreate the (compressed) image from provided codebook, pixel labels, image width, and image height

def recreate_image(codebook, labels, width, height):
    return codebook[labels].reshape(width, height, -1)

In [ ]:
# OPTIONAL: use this code (uncommented!) to see what the `codebook[labels]` code in the 'recreate_inage' function is doing
# test_codebook = make_codebook("orange", "navy")
# test_labels = np.array([0,1,1,0,1,0,0])
# test_codebook[test_labels]

In [ ]:
# Create and plot `SatArt` variable, a recreated reduced colour image of n_colours chosen colours as defined in the custom codebook colour palette
SatArt = recreate_image(codebook_6colours, labels, width, height)
plt.figure(figsize=(10,10))
plt.clf()
plt.axis("off")
plt.title(f"Satellite Art - a quantized image with {n_colours} colours")
plt.imshow(SatArt);
#plt.savefig("SatArt_test.png", dpi=600)

---
## ALTERNATIVE - up to 148 randomly-chosen colours

In [ ]:
# Choose the number of clusters, i.e. colours, that the image data will be grouped/reduced into
n_random_colours = 148

# Train K-Means clustering model on small sub-sample of the image data
print("Fitting model on a small sub-sample of the data")
t0_alt = time()
image_array_sample_alt = shuffle(image_array, random_state=0, n_samples=1_000)
kmeans_alt = KMeans(n_clusters=n_random_colours, n_init="auto", random_state=0).fit(
    image_array_sample_alt
)
print(f"done in {time() - t0_alt:0.3f}s.")

# Get labels for all points - use the kmeans trained model to predict which of the n clusters each pixel belongs to, i.e. assign each pixel a label, from 1 to n_colors
print("Predicting colour indices on the full image (k-means)")
t0_alt = time()
labels_alt = kmeans_alt.predict(image_array)
print(f"done in {time() - t0_alt:0.3f}s.")

In [ ]:
# Write function to recreate the (compressed) image from provided codebook, pixel labels, image width, and image height

def recreate_image_alt(codebook, labels, width, height):
    return codebook[labels_alt].reshape(width, height, -1)

In [ ]:
def random_array_rgb(n_random_colours):
    random_array = np.random.choice([i for i in mcolours.cnames.keys()], n_random_colours, replace=False)
    random_array_rgb = np.array([mcolours.to_rgb(i) for i in random_array])
    return (random_array_rgb, random_array)

In [ ]:
# Create and plot `SatArt_random` variable, a recreated reduced colour image of n_random_colours chosen colours as defined in the custom codebook colour palette

SatArt_random = recreate_image_alt(random_array_rgb(n_random_colours)[0], labels_alt, width, height)
plt.figure(figsize=(10,10))
plt.clf()
plt.axis("off")
plt.title(f"Satellite Art with {n_random_colours} colours: {random_array_rgb(n_random_colours)[1]}", fontdict={"fontsize":8})
plt.imshow(SatArt_random);
#plt.savefig("SatArt_test.png", dpi=600)

In [ ]:
# Convert 'SatArt_random' into RGB image object, scaling the 0-1 values and casting/conversion back to uint8/8 bit integer format
SatArt_random_uint8 = Image.fromarray((SatArt_random * 255).astype(np.uint8))

In [ ]:
# Save
#SatArt_random_uint8.save("SatArt_random.png")

---
### Now explore applying ready-made image filters to your Machine-Learning-enabled creation

In [ ]:
# Set-up
from PIL.ImageFilter import (FIND_EDGES, CONTOUR, EMBOSS, MedianFilter, MinFilter, MaxFilter)
from PIL.ImageOps import solarize

In [ ]:
# Create copy of the non-random ML SatArt object (or can copy 'SatArt_random') converted into RGB image object, scaling the 0-1 values and casting/conversion back to uint8/8 bit integer format
SatArt_ML = Image.fromarray((SatArt * 255).astype(np.uint8)).copy()

In [ ]:
# Check out what your 'SatArt' artwork looks like with an additional ready-made filter from the Python Imaging Library (PIL)

PIL_filters = ["FIND_EDGES", "FIND_EDGES (greyscale)", "CONTOUR", "CONTOUR (greyscale)", "EMBOSS", "MedianFilter", "MinFilter", "MaxFilter",  "solarize"]
fig, ax = plt.subplots(9,1, figsize=(40,80))

ax[0].imshow(SatArt_ML.filter(FIND_EDGES()))    
ax[1].imshow(SatArt_ML.convert("L").filter(FIND_EDGES()))    # Also try the FIND_EDGES filter on a greyscale version of your artwork
ax[2].imshow(SatArt_ML.filter(CONTOUR()))     
ax[3].imshow( SatArt_ML.convert("L").filter(CONTOUR()))    # Also try the CONTOUR filter on a greyscale version of your artwork

ax[4].imshow(SatArt_ML.filter(EMBOSS()))
ax[5].imshow(SatArt_ML.filter(MedianFilter(size=5)))
ax[6].imshow(SatArt_ML.filter(MinFilter(size=5)))
ax[7].imshow(SatArt_ML.filter(MaxFilter(size=5)))
ax[8].imshow(solarize(SatArt_ML, threshold=0))    # Try different values for the threshold argument between 0 and 128

for count, value in enumerate(PIL_filters):
    ax[count].axis("off")
    ax[count].set_title(f"{count+1}. Quantized image with {value} PIL filter")
plt.show()

In [ ]:
# NOTE: image objects can be saved by chaining the `.save()` function, inputting a string name - example below
# SatArt_ML.convert("L").filter(FIND_EDGES()).save("SatArt_PILfilters.png")

In [ ]:
# Can also generate a 'Before vs After' side-by-side comparison of the original chosen satellite image next to your final digital creation

fig, ax = plt.subplots(1,2, figsize=(20,20))
ax[0].imshow(raw_jpg)
ax[1].imshow((solarize(SatArt_ML, threshold=0)))
ax[0].axis("off")
ax[1].axis("off")

ax[0].set_title(f"Original Sastellite image with {original_n_colours} colours")
ax[1].set_title(f"Digital Satellite Art with {n_colours} colours")
#plt.savefig("SatArtComparison.png", dpi=600)
plt.show()

---
Copyright © 2023 Rho Zeta AI Ltd. All rights reserved.